In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amitshekhawat","key":"32f5f9d2ddb79757a5cedc21f9c5c453"}'}

In [3]:
# Make a directory for the Kaggle API key and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Install Kaggle CLI if not already installed
!pip install -q kaggle

# Download the ASL Alphabet dataset
!kaggle datasets download -d grassknoted/asl-alphabet

# Unzip the downloaded dataset
!unzip -q asl-alphabet.zip -d asl_data

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
 97% 1.00G/1.03G [00:07<00:00, 189MB/s]
100% 1.03G/1.03G [00:07<00:00, 141MB/s]


In [4]:
# Define dataset path and check classes
dataset_path = '/content/asl_data/asl_alphabet_train/asl_alphabet_train'
classes = sorted(os.listdir(dataset_path))
print("Number of classes:", len(classes))
print("Class names:", classes)

Number of classes: 29
Class names: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [5]:
# Enhanced data augmentation to reduce overfitting
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,           # Increased rotation
    zoom_range=0.2,             # Increased zoom
    width_shift_range=0.2,      # Increased shift
    height_shift_range=0.2,     # Increased shift
    shear_range=0.2,            # Added shear
    horizontal_flip=True,        # Added horizontal flip
    fill_mode='nearest'
)

In [6]:
# Training generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,              # Reduced batch size for better generalization
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [7]:
# Improved model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(256, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Added layer
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),

    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Increased units
    Dropout(0.5),
    Dense(29, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile with a custom learning rate
optimizer = Adam(learning_rate=0.0001)  # Lower learning rate for better convergence
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 930,013 (3.55 MB)

 Trainable params: 929,053 (3.54 MB)

 Non-trainable params: 960 (3.75 KB)

In [9]:
# Callbacks for early stopping and model checkpointing
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
]

In [10]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,  # Increased epochs with early stopping
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.1603 - loss: 11.5832

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 184s 81ms/step - accuracy: 0.1604 - loss: 11.5823 - val_accuracy: 0.3232 - val_loss: 7.0005
Epoch 2/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.4946 - loss: 5.5526

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 182s 84ms/step - accuracy: 0.4946 - loss: 5.5522 - val_accuracy: 0.5167 - val_loss: 3.9839
Epoch 3/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7047 - loss: 3.0147

...

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 171s 79ms/step - accuracy: 0.9099 - loss: 0.9260 - val_accuracy: 0.7521 - val_loss: 1.4668
Epoch 8/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 181s 83ms/step - accuracy: 0.9175 - loss: 0.8240 - val_accuracy: 0.7513 - val_loss: 1.3514
Epoch 9/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9262 - loss: 0.7442

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 192s 78ms/step - accuracy: 0.9262 - loss: 0.7442 - val_accuracy: 0.7717 - val_loss: 1.2546
Epoch 10/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9319 - loss: 0.6926

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 172s 79ms/step - accuracy: 0.9319 - loss: 0.6926 - val_accuracy: 0.7734 - val_loss: 1.3022
Epoch 11/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 169s 78ms/step - accuracy: 0.9362 - loss: 0.6526 - val_accuracy: 0.7698 - val_loss: 1.1942
Epoch 12/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9388 - loss: 0.6204

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 166s 76ms/step - accuracy: 0.9388 - loss: 0.6204 - val_accuracy: 0.8037 - val_loss: 1.0946
Epoch 13/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 168s 77ms/step - accuracy: 0.9432 - loss: 0.5911 - val_accuracy: 0.7597 - val_loss: 1.2717
Epoch 14/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 167s 77ms/step - accuracy: 0.9444 - loss: 0.5687 - val_accuracy: 0.7826 - val_loss: 1.1863
Epoch 15/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 166s 77ms/step - accuracy: 0.9476 - loss: 0.5531 - val_accuracy: 0.7889 - val_loss: 1.1092
Epoch 16/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 177s 81ms/step - accuracy: 0.9476 - loss: 0.5420 - val_accuracy: 0.8018 - val_loss: 1.0373
Epoch 17/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9473 - loss: 0.5359

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 170s 78ms/step - accuracy: 0.9473 - loss: 0.5359 - val_accuracy: 0.8153 - val_loss: 0.9695
Epoch 18/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 178s 82ms/step - accuracy: 0.9527 - loss: 0.5142 - val_accuracy: 0.8095 - val_loss: 1.0418
Epoch 19/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 170s 78ms/step - accuracy: 0.9522 - loss: 0.5110 - val_accuracy: 0.7724 - val_loss: 1.1619
Epoch 20/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 178s 82ms/step - accuracy: 0.9507 - loss: 0.5074 - val_accuracy: 0.7220 - val_loss: 1.4548
Epoch 21/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 168s 77ms/step - accuracy: 0.9554 - loss: 0.5007 - val_accuracy: 0.7988 - val_loss: 1.0469
Epoch 22/30
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 168s 77ms/step - accuracy: 0.9562 - loss: 0.4844 - val_accuracy: 0.7798 - val_loss: 1.2840


In [11]:
model.save("sign_language_model_final.h5")